# COGS 108 - Data Checkpoint

# Names

- Angkitha Anguraj
- Christian Goodale
- Immanuel Tran
- Camille Yabut
- Sophia Yonus

<a id='research_question'></a>
# Research Question

Is there a relationship between the **number of new enrollees** in the University of California (UC) schools and the **rental prices** of the county where the UC school is located in the years 2010-2021? Additionally, what other factors, such as college ranking and median household income, affect the location’s rental price? 

# Dataset(s)

**1. Admission Data**
- Dataset Name: Undergraduate admissions summary
- Link to Dataset: https://www.universityofcalifornia.edu/about-us/information-center/admissions-residency-and-ethnicity 
- Number of observations: 1 dataset per UC (9 total), 24x11 observations in each set

This dataset contains the undergraduate admissions data for all UC schools from the University of California itself. We are specifically interested in the new number of enrollees, defined as “students who have accepted an offer of admission and are enrolled at the University of California.”  

**2. Median Rental Prices**
- Dataset Name: B25064 MEDIAN GROSS RENT (DOLLARS)
- Link to Dataset: https://data.census.gov/cedsci/table?q=B25064%3A%20MEDIAN%20GROSS%20RENT%20%28DOLLARS%29&g=0100000US_0400000US06_0500000US06001,06037,06047,06059,06065,06073,06083,06087,06113&tid=ACSDT1Y2021.B25064&moe=true 
- Number of observations: 1 row per year 

This dataset from the US Census American Community Survey describes the median gross rent in dollars filtered by the 9 counties where the UC schools are located. As each dataset is separated by year, we will download the separate CSV files and concatenate the rows to create a data frame containing all the rent data from 2010-2021. 

**3. College Ranking**  

Dataset Name & Description
- U.S. News: America’s Top National Universities from the UC Accountability Reports
- These datasets are obtained from the University of California’s Accountability Annual Reports. They list all of the UCs national rankings by place competing with private and public universities. We plan to look at the table for America’s Top National Universities from 2010-2021 for our data analysis. 

How We Plan to Combine the Data Sets
- To get this data, we will use web scraping to pull out the table of interest – U.S. News: America’s Top National Universities – for each year. Afterward, we will concatenate the data into one data frame to pull out the desired rankings. 

Links to the Dataset & Number of Observations
- 2010-2017 Rankings
    - https://accountability.universityofcalifornia.edu/2017/chapters/chapter-14.html 
    - 17 rows, only 9 relevant to analysis 
- 2018 Rankings
    - https://accountability.universityofcalifornia.edu/2018/chapters/chapter-13.html 
    - 9 rows
- 2019 Rankings
    - https://accountability.universityofcalifornia.edu/2019/chapters/chapter-13.html 
    - 9 rows
- 2020 Rankings
    - https://accountability.universityofcalifornia.edu/2020/chapters/chapter-13.html 
    - 9 rows
- 2021 Rankings
    - https://accountability.universityofcalifornia.edu/2021/chapters/chapter-13.html 
    - 9 rows

**4. Median Household Income (by County)**

Dataset Names & Description 
- FRED St. Louis Fed Data:
    - These datasets gather an estimate of the median household income (dollars) for a specified county for the years ~1989-2020. This dataset only includes two columns, the DATE which is the 1st of January for the years 1989-2020, and the MHICAXXXXXXXXX which is the Median Household Income in dollars. We gathered the datasets for the relevant counties.
- 2021 ACS Data:
    - This Dataset comes from the American Community Survey and it gathers information on various different economic characteristics for American households in each county of each state. It measures aspects of employment, occupation, health insurance, income bracket, and measures of central tendency for household income, and gets totals and percentages for each measured variable where possible. We only want to use the Median Household Income (dollars) for the relevant communities. 

How We Plan to Combine the Data Sets
- The St. Louis Fed datasets only measure one county for each set so we will have to merge the 9 datasets together into one data frame matching based on year. We will need to use the ACS dataset to add the median household income for the year 2021 since it isn’t accounted for in our other data. Prior to adding this as another row to our data frame, we checked the estimates ACS gave for other years to see if they were not notably different enough to possibly throw off our analysis.

Link to Data Sets and Number of Observations 
- 2021 All Counties Data
    - American Community Survey | DP03 | Selected Economic Characteristics (2021)
    - https://data.census.gov/cedsci/table?t=Families%20and%20Household%20Characteristics&g=0500000US06001,06037,06047,06059,06065,06073,06083,06087,06113&d=ACS%201-Year%20Estimates%20Data%20Profiles&tid=ACSDP1Y2021.DP03&moe=false
    - ~140x18, only 9 observations are relevant to our analysis
- UC Berkeley (Alameda County)
    - https://fred.stlouisfed.org/series/MHICA06001A052NCEN 
    - 30x2, only 20 relevant 
- UC Los Angeles (Los Angeles County)
    - https://fred.stlouisfed.org/series/MHICA06037A052NCEN 
    - 30x2, only 20 relevant
- UC San Diego (San Diego County)
    - https://fred.stlouisfed.org/series/MHICA06073A052NCEN 
    - 30x2, only 20 relevant
- UC Santa Barbara
    - https://fred.stlouisfed.org/series/MHICA06083A052NCEN 
    - 30x2, only 20 relevant
- UC Irvine (Orange County)
    - https://fred.stlouisfed.org/series/MHICA06059A052NCEN 
    - 30x2, only 20 relevant
- UC Riverside (Riverside County)
    - https://fred.stlouisfed.org/series/MHICA06065A052NCEN 
    - 30x2, only 20 relevant
- UC Santa Cruz (Santa Cruz)
    - https://fred.stlouisfed.org/series/MHICA06087A052NCEN 
    - 30x2, only 20 relevant
- UC Davis (Yolo County)
    - https://fred.stlouisfed.org/series/MHICA06113A052NCEN 
    - 30x2, only 20 relevant
- UC Merced (Merced County)
    - https://fred.stlouisfed.org/series/MHICA06047A052NCEN 
    - 30x2, only 20 relevant

# Setup

In [1]:
#Imports
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 104)
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

# Data Cleaning

## 1. Admissions Data

In [2]:
#The admissions data breaks down the rates of Applicants, Admits, and Enrolled by Ethnicity per year
#Since we're looking only for those entering the school we will look under the "Enrollees"
#We also don't need to break down by ethnicity so we will look at the observation for "All" which is a sum for each

#To start the cleaning process we will need to slice the data to only get "All" in the "Enrollees" section for each year
#Once we have those observations isolated for each college we will need to put them all into a single dataframe
#To do this we will merge the dataframes using pd.merge() matching by Year

#To finish cleaning we will need to make sure that the columns reflect the correct UC and are named in a way that is understandable
#We will also need to make sure the Years are listed correctly
#Lastly, we will make sure the data points are all the correct type (integers) and that there are no missing values

#As a note for later: we will need to find the 2010 admissions data


## 2. Median Rent Data

We cleaned up the CSV files by removing unneccessary columns, renaming the columns, and adding in the year for each data set.

In [3]:
# read csv file
rent_2021 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2021.csv')

# drop columns we do not need
rent_2021 = rent_2021.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2021 = rent_2021.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2021['Year'] = ['2021']

# display cleaned data set
rent_2021;

In [4]:
# read csv file
rent_2020 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT5Y2020.csv')

# drop columns we do not need
rent_2020 = rent_2020.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2020 = rent_2020.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2020['Year'] = ['2020']

# display cleaned data set
rent_2020;

In [5]:
# read csv file
rent_2019 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2019.csv')

# drop columns we do not need
rent_2019 = rent_2019.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2019 = rent_2019.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2019['Year'] = ['2019']

# display cleaned data set
rent_2019;

In [6]:
# read csv file
rent_2018 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2018.csv')

# drop columns we do not need
rent_2018 = rent_2018.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2018 = rent_2018.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2018['Year'] = ['2018']

# display cleaned data set
rent_2018;

In [7]:
# read csv file
rent_2017 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2017.csv')

# drop columns we do not need
rent_2017 = rent_2017.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2017 = rent_2017.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2017['Year'] = ['2017']

# display cleaned data set
rent_2017;

In [8]:
# read csv file
rent_2016 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2016.csv')

# drop columns we do not need
rent_2016 = rent_2016.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2016 = rent_2016.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2016['Year'] = ['2016']

# display cleaned data set
rent_2016;

In [9]:
# read csv file
rent_2015 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2015.csv')

# drop columns we do not need
rent_2015 = rent_2015.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2015 = rent_2015.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2015['Year'] = ['2015']

# display cleaned data set
rent_2015;

In [10]:
# read csv file
rent_2014 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2014.csv')

# drop columns we do not need
rent_2014 = rent_2014.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2014 = rent_2014.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2014['Year'] = ['2014']

# display cleaned data set
rent_2014;

In [11]:
# read csv file
rent_2013 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2013.csv')

# drop columns we do not need
rent_2013 = rent_2013.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2013 = rent_2013.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2013['Year'] = ['2013']

# display cleaned data set
rent_2013;

In [12]:
# read csv file
rent_2012 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2012.csv')

# drop columns we do not need
rent_2012 = rent_2012.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2012 = rent_2012.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2012['Year'] = ['2012']

# display cleaned data set
rent_2012;

In [13]:
# read csv file
rent_2011 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2011.csv')

# drop columns we do not need
rent_2011 = rent_2011.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2011 = rent_2011.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2011['Year'] = ['2011']

# display cleaned data set
rent_2011;

In [14]:
# read csv file
rent_2010 = pd.read_csv('https://raw.githubusercontent.com/cpyabut/Data-Sets/main/ACSDT1Y2010.csv')

# drop columns we do not need
rent_2010 = rent_2010.drop(columns=['United States!!Estimate',
                       'United States!!Margin of Error', 
                       'California!!Estimate',
                       'California!!Margin of Error', 
                       'Alameda County, California!!Margin of Error', 
                       'Los Angeles County, California!!Margin of Error', 
                       'Merced County, California!!Margin of Error',
                       'Orange County, California!!Margin of Error', 
                       'Riverside County, California!!Margin of Error',
                       'San Diego County, California!!Margin of Error',
                       'Santa Barbara County, California!!Margin of Error',
                       'Santa Cruz County, California!!Margin of Error', 
                       'Yolo County, California!!Margin of Error'])

# rename columns for better understanding 
rent_2010 = rent_2010.rename(columns={
    'Label (Grouping)':'Year',
    'Alameda County, California!!Estimate':'Alameda County', 
    'Los Angeles County, California!!Estimate':'Los Angeles County',
    'Merced County, California!!Estimate':'Merced County',
    'Orange County, California!!Estimate':'Orange County',
    'Riverside County, California!!Estimate':'Riverside County',
    'San Diego County, California!!Estimate':'San Diego County',
    'Santa Barbara County, California!!Estimate':'Santa Barbara County',
    'Santa Cruz County, California!!Estimate':'Santa Cruz County',
    'Yolo County, California!!Estimate':'Yolo County'
    })

# add year data into year column
rent_2010['Year'] = ['2010']

# display cleaned data set
rent_2010;

Then, concatenated all the different datasets into one data frame. 

In [15]:
rent_data = pd.concat([rent_2010, 
                       rent_2011, 
                       rent_2012, 
                       rent_2013, 
                       rent_2014, 
                       rent_2015,
                       rent_2016,
                       rent_2017,
                       rent_2018,
                       rent_2019, 
                       rent_2020, 
                       rent_2021], ignore_index=True)

rent_data

,Year,Alameda County,Los Angeles County,Merced County,Orange County,Riverside County,San Diego County,Santa Barbara County,Santa Cruz County,Yolo County
0,2010,"1,198","1,147",807,"1,402","1,121","1,249","1,267","1,284","1,056"
1,2011,"1,244","1,161",814,"1,430","1,101","1,237","1,265","1,358",987
2,2012,"1,265","1,175",834,"1,465","1,129","1,253","1,324","1,474","1,011"
3,2013,"1,335","1,205",864,"1,499","1,164","1,289","1,336","1,350","1,110"
4,2014,"1,391","1,239",858,"1,572","1,196","1,373","1,395","1,477","1,094"
5,2015,"1,513","1,279",874,"1,624","1,195","1,427","1,419","1,442","1,160"
6,2016,"1,622","1,330",899,"1,722","1,275","1,504","1,542","1,594","1,210"
7,2017,"1,745","1,402","1,003","1,786","1,313","1,598","1,587","1,680","1,231"
8,2018,"1,875","1,479","1,023","1,872","1,374","1,668","1,653","1,685","1,320"
9,2019,"1,982","1,577","1,097","1,929","1,497","1,758","1,660","1,785","1,489"


Finally, change data type to integer. 

In [16]:
# will reassign object to int data type for each column

## 3. College Ranking Data

In [17]:
# webscrape each site for desired table
# isolate data points of interest
# combine data sets into one dataframe 

## 4. Median Household Income Data

In [18]:
#defining a function to do cleaning procedures on a CSV 's'
def clean_income(s):
    
    df = pd.read_csv(s) #Read in dataset
    
    year = pd.to_datetime(df['DATE']) #Keep only year from date column
    year.dt.year
    df['DATE']= year.dt.year
    
    rel_years = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020] #Slice only relevant years
    df = df[df.DATE.isin(rel_years)].reset_index()
    
    df = df.rename(columns = {'DATE':"Year"}) #Rename Date column to Year
    
    return df

In [19]:
#Read in and clean FRED datasets for all 9 counties 2010-2020
Alameda_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06001A052NCEN.csv')
LosAngeles_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06037A052NCEN.csv')
Merced_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06047A052NCEN.csv')
Orange_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06059A052NCEN.csv')
Riverside_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06065A052NCEN.csv')
SanDiego_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06073A052NCEN.csv')
SantaBarbara_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06083A052NCEN.csv')
SantaCruz_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06087A052NCEN.csv')
Yolo_df = clean_income('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/MHICA06113A052NCEN.csv')

In [20]:
#Merge all FRED datasets into one matching based on year
merge1 = pd.merge(Alameda_df, LosAngeles_df[['Year', 'MHICA06037A052NCEN']], on= 'Year')
merge2 = pd.merge(merge1, Merced_df[['Year', 'MHICA06047A052NCEN']], on= 'Year')
merge3 = pd.merge(merge2, Orange_df[['Year', 'MHICA06059A052NCEN']], on= 'Year')
merge4 = pd.merge(merge3, Riverside_df[['Year', 'MHICA06065A052NCEN']], on= 'Year')
merge5 = pd.merge(merge4, SanDiego_df[['Year', 'MHICA06073A052NCEN']], on= 'Year')
merge6 = pd.merge(merge5, SantaBarbara_df[['Year', 'MHICA06083A052NCEN']], on= 'Year')
merge7 = pd.merge(merge6, SantaCruz_df[['Year', 'MHICA06087A052NCEN']], on= 'Year')
merge_all_df = pd.merge(merge7, Yolo_df[['Year', 'MHICA06113A052NCEN']], on= 'Year')

In [21]:
#Rename columns so they respond to county names
Counties_dict = {'MHICA06001A052NCEN': 'Alameda',
                'MHICA06037A052NCEN': 'Los Angeles',
                'MHICA06047A052NCEN': 'Merced',
                'MHICA06059A052NCEN': 'Orange',
                'MHICA06065A052NCEN': 'Riverside',
                'MHICA06073A052NCEN': 'San Diego',
                'MHICA06083A052NCEN': 'Santa Barbara',
                'MHICA06087A052NCEN': 'Santa Cruz',
                'MHICA06113A052NCEN': 'Yolo'}
Counties_income = merge_all_df.rename(columns = Counties_dict)

In [22]:
#Drop index column
Counties_income = Counties_income.drop(columns = ['index'])

In [23]:
#Read in data for 2021 from American Community Survey

data2021 = pd.read_csv('https://raw.githubusercontent.com/cgoodale01/Median-Household-Income-Data/main/ACSDP1Y2021.DP03-2022-11-07T044023.csv')
df2021 = pd.DataFrame(data2021)

In [24]:
#Clean data only taking the estimate of Median Household Income for each county
important_col = ['Alameda County, California!!Estimate',
                 'Los Angeles County, California!!Estimate',
                 'Merced County, California!!Estimate',
                'Orange County, California!!Estimate',
                'Riverside County, California!!Estimate',
                'San Diego County, California!!Estimate',
                'Santa Barbara County, California!!Estimate',
                'Santa Cruz County, California!!Estimate',
                'Yolo County, California!!Estimate']

#Slicing only the estimate columns and not percents or margin of error and take only the Median Household Income row
#Add column for Year to match it with the 2010-2020
#Rename county columns to match with 2010-2020 data

df2021 = df2021[important_col]
df2021_income = df2021.iloc[67:68,:]
df2021_income['Year']= 2021
df2021_income = df2021_income.rename(columns = {'Alameda County, California!!Estimate':'Alameda',
                 'Los Angeles County, California!!Estimate':'Los Angeles',
                 'Merced County, California!!Estimate': 'Merced',
                'Orange County, California!!Estimate': 'Orange',
                'Riverside County, California!!Estimate': 'Riverside',
                'San Diego County, California!!Estimate': 'San Diego',
                'Santa Barbara County, California!!Estimate': 'Santa Barbara',
                'Santa Cruz County, California!!Estimate':'Santa Cruz',
                'Yolo County, California!!Estimate':'Yolo'})

In [25]:
#Append the 2021 Year data to our data for 2010-2020 and reset index
Counties_income = Counties_income.append(df2021_income, )
Counties_income = Counties_income.reset_index()
Counties_income = Counties_income.drop(columns = ['index'])

In [26]:
#Convert median household income values to integers
for x in range (1,10):
    Counties_income.iloc[11,x] = Counties_income.iloc[11,x].replace(',','')

for i in range (0,12):
    for j in range (1,10):
        Counties_income.iloc[i,j] = int(Counties_income.iloc[i,j])

In [27]:
#Check data
Counties_income

,Year,Alameda,Los Angeles,Merced,Orange,Riverside,San Diego,Santa Barbara,Santa Cruz,Yolo
0,2010,66937,52595,41730,70727,53981,59759,56243,60247,54433
1,2011,67295,52239,40016,72046,52491,59290,59494,61228,50174
2,2012,70209,52929,42552,71866,52416,60235,60683,65799,51040
3,2013,72128,54443,41003,73827,53909,61365,60803,65282,55011
4,2014,76996,55686,43818,76061,56877,66034,62116,64257,54509
5,2015,81462,59045,42879,78002,57895,67053,63049,64841,58766
6,2016,89472,61308,47442,81642,59951,70693,66360,75929,63645
7,2017,95550,64912,48036,86031,63776,76048,70651,76633,64900
8,2018,101744,67986,56169,89373,66793,78777,75646,84213,67804
9,2019,107589,72721,59733,95761,72905,83576,74530,85770,70951
